In [ ]:
(asdf:load-system :cram-pr2-pick-place-demo)
(roslisp-utilities:startup-ros)
(defpackage cram-bullet-world-tutorial
  (:nicknames :btw-tut)
  (:use :common-lisp :cram-prolog :desig :exe))
(in-package :btw-tut)

(defun make-pose (frame point euler)
    (declare (type string frame)
             (type list point euler))
    "Creates a cl-transforms pose in 3D space w.r.t. the given frame, point and euler."
    (if (and (eq (length point) 3) (eq (length euler) 3))
        (destructuring-bind (ax ay az) (mapcar #'eval euler)
          (cl-transforms-stamped:make-pose-stamped frame 0.0 
                                                   (apply #'cl-transforms:make-3d-vector point)
                                                   (cl-transforms:euler->quaternion :ax ax :ay ay :az az)))
        (format T "point ~a or euler ~a is not of length 3" point euler)))

(defun simulate (&key (duration 5.0)    ;; in seconds 
                      (framerate 20.0)) ;; in Hz
    (let ((sleep-interval (/ 1.0 framerate))
          (times (floor (* duration framerate))))
         (dotimes (_ times)
             (sleep (/ sleep-interval 2))
             (btr:simulate btr:*current-bullet-world* (/ sleep-interval 2)))))

(defparameter *bottle-spawn-pose* (make-pose "map" '(-2.323 -1 0.82) '(0 0 0)))
(defparameter *base-pose-near-table* (make-pose "map" '(-2.2 -0.2 0.0) '(0.0 0.0 (* pi -0.5))))
(defparameter *downward-look-coordinate* (make-pose "base_footprint" '(0.7 0.0 1.0) '(0 0 0)))
(defparameter *object-delivery-pose* (make-pose "map" '(-0.8 2 0.9) '(0 0 0)))
(defparameter *base-pose-near-kitchen-island* (make-pose "map" '(-0.15 2.0 0.0) '(0.0 0.0 pi)))

In [ ]:
(defun pick (?object ?nav-goal ?poi &key ?arm)
    (let ((?arm (if (or (listp ?arm) (not ?arm))
                    ?arm
                    `(,?arm)))
          ?perceived-object)
       (cpl:par
             ;; Park arms
             (perform (an action
                          (type parking-arms)))
             ;; Move the torso up
             (perform (a motion 
                         (type moving-torso)
                         (joint-angle 0.3)))
             ;; Go to the navigation goal
             (perform (an action
                          (type going)
                          (target (a location 
                                     (pose ?nav-goal))))))
         ;; Look at the point of interest
         (perform (an action
                      (type looking)
                      (target (a location 
                                 (pose ?poi)))))
         ;; Detect the object
         (setf ?perceived-object 
               (perform (an action
                            (type detecting)
                            (object ?object))))
         ;; Pick up the detected object
         (perform (an action
                      (type picking-up)
                      (desig:when ?arm
                          (arm ?arm))
                      (object ?perceived-object)))
         ;; Park arms
         (perform (an action
                      (type parking-arms)))
         ?perceived-object))

In [ ]:
(defun place (?nav-goal ?drop-pose &key ?object ?arm)
    (let ((?arm (if (or (listp ?arm) 
                        (not ?arm))
                    ?arm
                    `(,?arm))))
         ;; Move the torso up so the robot can reach the higher surface better
         ;; without crashing his arms into it.
         (perform (a motion 
                     (type moving-torso)
                     (joint-angle 0.3)))
         ;; Go to the target delivery location
         (perform (an action
                      (type going)
                      (target (a location
                                 (pose ?nav-goal)))))
         ;; Place down the bottle
         (perform (an action
                      (type placing)
                      (desig:when ?arm
                          (arm ?arm))
                      (desig:when ?object
                          (object ?object))
                      (target (a location
                                 (pose ?drop-pose)))))
         ;; Park arms
         (perform (an action
                      (type parking-arms)))))

In [ ]:
;; original
;; (defparameter *object-delivery-pose* (make-pose "map" '(-0.8 2 0.9) '(0 0 0)))

;; higher up
(defparameter *object-delivery-pose* (make-pose "map" '(-0.9 2 1.1) '(0 0 0)))

;; sideways
;; (defparameter *object-delivery-pose* (make-pose "map" '(-0.8 2 1.2) '((* 1.5 pi) (* 0.5 pi) pi)))


In [ ]:
;; Reset the world
(demo::initialize)

(loop while t
    do 
      (btr-utils:kill-all-objects)
    ;; Spawn the bottle
    (btr-utils:spawn-object 'bottle-1 :bottle :pose *bottle-spawn-pose*)
    
    (urdf-proj:with-simulated-robot
     (let ((?drop-pose *object-delivery-pose*)
           ?perceived-object)

          (pick (an object (type bottle))
                *base-pose-near-table*
                *downward-look-coordinate*
                :?arm :right)

          (place *base-pose-near-kitchen-island*
                 *object-delivery-pose*))) 

        (simulate))

In [ ]:
;; Reset the world
(demo::initialize)

;; Have all parameters we need in one place.
(defparameter *bottle-spawn-pose* (make-pose "map" '(-2.323 -1 0.82) '(0 0 0)))
(defparameter *base-pose-near-table* (make-pose "map" '(-2.2 -0.2 0.0) '(0.0 0.0 (* pi -0.5))))
(defparameter *downward-look-coordinate* (make-pose "base_footprint" '(0.7 0.0 1.0) '(0 0 0)))
(defparameter *object-delivery-pose* (make-pose "map" '(-0.8 2 0.9) '(0 0 0)))
(defparameter *base-pose-near-kitchen-island* (make-pose "map" '(-0.15 2.0 0.0) '(0.0 0.0 pi)))

;; Spawn the bottle
(btr-utils:spawn-object 'bottle-1 :bottle :pose *bottle-spawn-pose*)

(urdf-proj:with-simulated-robot

 (let ((?nav-goal *base-pose-near-kitchen-island*)
       (?looking-pose *downward-look-coordinate*)
       (?drop-pose *object-delivery-pose*)
       ?perceived-bottle)
      
      (setf ?perceived-bottle
            (pick (an object (type bottle))
                  *base-pose-near-table*
                  *downward-look-coordinate*))
               
      
      ;; Move the torso up so the robot can reach the higher surface better
      ;; without crashing his arms into it.
      (perform (a motion 
                  (type moving-torso)
                  (joint-angle 0.3)))
      ;; Go to the target delivery location
      (perform (an action
                   (type going)
                   (target (a location
                              (pose ?nav-goal)))))
      ;; Place down the bottle
      (perform (an action
                   (type placing)
                   (arm (right))
                   ;; here we explicitly specify to put down the bottle
                   ;; it's not necessary, but you can use this as a reference for the upcoming task
                   (object ?perceived-bottle) 
                   (target (a location
                              (pose ?drop-pose)))))
            ;; Park arms
      (perform (an action
                   (type parking-arms)))))

In [ ]:
;; Reset the world
(demo::initialize)

;; Have all parameters we need in one place.
(defparameter *bottle-spawn-pose* (make-pose "map" '(-2.323 -1 0.82) '(0 0 0)))
(defparameter *base-pose-near-table* (make-pose "map" '(-2.2 -0.2 0.0) '(0.0 0.0 (* pi -0.5))))
(defparameter *downward-look-coordinate* (make-pose "base_footprint" '(0.7 0.0 1.0) '(0 0 0)))
(defparameter *object-delivery-pose* (make-pose "map" '(-0.8 2 0.9) '(0 0 0)))
(defparameter *base-pose-near-kitchen-island* (make-pose "map" '(-0.15 2.0 0.0) '(0.0 0.0 pi)))

;; Spawn the bottle
(btr-utils:spawn-object 'bottle-1 :bottle :pose *bottle-spawn-pose*)

(urdf-proj:with-simulated-robot
 ;; Go to the table where the bottle stands
 (let ((?navigation-goal *base-pose-near-table*))
      (perform (an action
                 (type going)
                 (target (a location 
                            (pose ?navigation-goal))))))
 ;; Look down at the bottle
 (let ((?looking-direction *downward-look-coordinate*))
      (perform (an action
                   (type looking)
                   (target (a location 
                              (pose ?looking-direction))))))
 ;; Detect the bottle and keep the object designator in ?perceived-bottle
 ;; so we can use it later
 (let ((?nav-goal *base-pose-near-kitchen-island*)
       (?drop-pose *object-delivery-pose*)
       (?perceived-bottle (perform (an action
                                       (type detecting)
                                       (object (an object
                                                   (type bottle)))))))
      ;; Pick up the detected bottle
      (perform (an action
                   (type picking-up)
                   (arm (right))
                   (object ?perceived-bottle)))
      ;; Park arms
      (perform (an action
                   (type parking-arms)))
      ;; Move the torso up so the robot can reach the higher surface better
      ;; without crashing his arms into it.
      (perform (a motion 
                  (type moving-torso)
                  (joint-angle 0.3)))
      ;; Go to the target delivery location
      (perform (an action
                   (type going)
                   (target (a location
                              (pose ?nav-goal)))))
      ;; Place down the bottle
      (perform (an action
                   (type placing)
                   (arm (right))
                   ;; here we explicitly specify to put down the bottle
                   ;; it's not necessary, but you can use this as a reference for the upcoming task
                   (object ?perceived-bottle) 
                   (target (a location
                              (pose ?drop-pose)))))
            ;; Park arms
      (perform (an action
                   (type parking-arms)))))

In [ ]:



(in-package :cram-bullet-reasoning-belief-state)


(defmethod cram-occasions-events:on-event btr-detach-object 2 ((event cpoe:object-detached-robot))
  (let* ((robot-object (btr:get-robot-object))
         (environment-object (btr:get-environment-object))
         (btr-object-name (cpoe:event-object-name event))
         (first-arm (first (cpoe:event-arm event)))
         (second-arm (second (cpoe:event-arm event)))
         (first-link (when first-arm
                       (if (get-ee-link first-arm)
                           (get-ee-link first-arm)
                           (if (cpoe:event-link event)
                               (cpoe:event-link event)
                               (error "[BTR-BELIEF OBJECT-DETACHED] either link or arm ~
                                 in object-attached-robot even had to be given...")))))
         (second-link (when second-arm
                        (if (get-ee-link second-arm)
                            (get-ee-link second-arm)
                            (if (cpoe:event-link event)
                                (cpoe:event-link event)
                                (error "[BTR-BELIEF OBJECT-DETACHED] either link or arm ~
                                  in object-attached-robot even had to be given..."))))))
    (when (and (cut:is-var first-link)
               (cut:is-var second-link))
      (error "[BTR-BELIEF OBJECT-DETACHED] Couldn't find robot's EE link."))
    (if btr-object-name
        ;; if btr-object-name was given, detach it from the robot link
        (let ((btr-object (btr:object btr:*current-bullet-world* btr-object-name)))
          (when btr-object
            (btr:detach-object robot-object btr-object :link first-link)
            (btr:detach-object robot-object btr-object :link second-link)
            (btr:simulate btr:*current-bullet-world* 0.1)
            ;; find the links and items that support the object
            ;; and attach the object to them.
            ;; links get proper attachments and items loose attachments
            (let ((contacting-links
                    (remove-duplicates
                     (mapcar
                      #'cdr
                      (remove-if-not
                       ;; filter all the links contacting items to our specific item
                       (lambda (item-and-link-name-cons)
                         (equal
                          (btr:name (car item-and-link-name-cons))
                          btr-object-name))
                       ;; get all links contacting items in the environment
                       (btr:link-contacts environment-object)))
                     :test #'equal))
                  ;; (contacting-items
                  ;;   (remove-if-not
                  ;;    (lambda (c) (typep c 'btr:item))
                  ;;    (btr:find-objects-in-contact btr:*current-bullet-world* btr-object)))
                  )
              ;; If btr-object is in contact with an item,
              ;; it will be attached loose.
              ;; Otherwise, if a link contacting btr-object was found,
              ;; btr-object will be attached to it.
              (or ;;-- (mapcar (lambda (item-object)
                  ;;           (when item-object
                  ;;             (btr:attach-object item-object btr-object :loose T)))
                  ;;         contacting-items)
                  (mapcar (lambda (link-name)
                            (btr:attach-object
                             environment-object btr-object :link link-name))
                          contacting-links)
                  (roslisp:ros-warn (btr-belief btr-detach-object)
                                    "Object ~a was detached from robot,
                                     but after falling down it
                                     is in no contact with the
                                     environment or another object."
                                    btr-object-name)))))
        ;; if btr-object-name was not given, detach all objects from the robot link
        (progn
          (btr:detach-all-from-link robot-object first-link)
          (btr:detach-all-from-link robot-object second-link)
          (btr:simulate btr:*current-bullet-world* 0.1)))))


(in-package :btw-tut)